### Recommendation System

In this project:
    
    1) Developed a music recommendation system for the users based on their previous FM playlist
    (Link to Dataset: http://www.dtic.upf.edu/%7Eocelma/MusicRecommendationDataset/lastfm-360K.html)
    No of records taken : 100,000 samples
    2) This was coded using a LightFM package

    3) Output prints the top 3 artist recommendations for the user

In [27]:
#Importing the libraries and dependencies

import os.path
from scipy.sparse import coo_matrix
import numpy as np
from lightfm import LightFM

#Function to fetch the datasets with minimum play count "200"


def fetch_data(min_plays=200):
   
    data_path = 'Desktop/100k_lines_lastfm.tsv'


    # Data to create our coo_matrix
    data, row, col = [], [], []

    # Artists by id, and users
    artists, users = {}, {}

    # Read the file and fill variables with data to
    # create the matrix and have the artists by id
    with open(data_path) as data_file:
        for n, line in enumerate(data_file):

            # If you use the original data from lastfm (14 million lines)
            # if n == SOMEINT: break
            
            # Readable data (for humans)
            readable_data = line.split('\t')
            user =           readable_data[0]
            artist_id =      readable_data[1]
            artist_name =    readable_data[2]
            plays =     int(readable_data[3])

            if user not in users:
                users[user] = len(users)

            if artist_id not in artists:
                artists[artist_id] = {
                        'name' : artist_name,
                        'id' : len(artists)
                        }

            # Data for the coo_matrix if the artist was played > 200 times
            if plays > min_plays:
                data.append(plays)
                row.append(users[user])
                col.append(artists[artist_id]['id'])

    # Our matrix: ((plays, (user, artist)))
    coo = coo_matrix((data,(row,col)))

    # We return the matrix, the artist dictionary and the amount of users
    dictionary = {
        'matrix' : coo,
        'artists' : artists,
        'users' : len(users)
    }

    return dictionary

The below block of code handles three functionalities
    1. Loading the data 
    2. Training the recommendation system model
    3. Obtain the recommendations 

In [28]:
# Loading the data
data = fetch_lastfm()

# Training the recommendation system model
model = LightFM(loss='warp')
model.fit(data['matrix'], epochs=30, num_threads=2)

# Get recommendationns function
def get_recommendations(model, coo_mtrx, users_ids):

    n_items = coo_mtrx.shape[1]

    for user in users_ids:

        # TODO create known positives
        # Artists the model predicts they will like
        scores = model.predict(user, np.arange(n_items))
        top_scores = np.argsort(-scores)[:3]

        print ('Recomendations for user %s:' % user)

        for x in top_scores.tolist():
            for artist, values in data['artists'].items():
                if int(x) == values['id']:
                    print ('   - %s' % values['name'])

        print ('\n') # Get it pretty


user_1 = input('Select user_1 (0 to %s): ' % data['users'])
user_2 = input('Select user_2 (0 to %s): ' % data['users'])
user_3 = input('Select user_3 (0 to %s): ' % data['users'])
print ('\n') # Get it pretty

get_recommendations(model, data['matrix'], [user_1, user_2, user_3])



Select user_1 (0 to 2033): 10
Select user_2 (0 to 2033): 43
Select user_3 (0 to 2033): 123


Recomendations for user 10:
   - coldplay
   - air
   - the killers


Recomendations for user 43:
   - bob marley
   - the clash
   - misfits


Recomendations for user 123:
   - rock universal
   - the beatles
   - radiohead


